# CSE 4633/6633 HW 1

The assignment for this notebook is to implement local search algorithms such as hill-climbing, random restart, and simulated annealing to perform function optimization.

In [63]:
# import required libraries
import time
import random
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

In [64]:
# Create a complex objective function that we need to maximize.
# Here, the objective function is simply defined as an equation but
# more generally, it could represent natural phenomena, physical laws, or mathematical models
#   X: a 2-dimensional floating-point vector consisting of an x-value and a y-value
#   returns: a scalar floating-point value representing the output of the objective function
def objective_function(X):
    x = X[0]
    y = X[1]
    value = 3 * (1 - x) ** 2 * math.exp(-x ** 2 - (y + 1) ** 2) - 10 * (x / 5 - x ** 3 - y ** 5) * math.exp(
        -x ** 2 - y ** 2) - (1 / 3) * math.exp(-(x + 1) ** 2 - y ** 2)
    return value

# to limit the search space for this problem, we will only consider solutions
# where x ranges from -4 to 4 and y ranges from -4 to 4
lower_bounds = [-4, -4]
upper_bounds = [4, 4]

A 3D plot of the objective function can be viewed [here](https://c3d.libretexts.org/CalcPlot3D/index.html?type=z;z=3(1-x)%5E2*exp(-x%5E2-(y+1)%5E2)-10(x/5-x%5E3-y%5E5)*exp(-x%5E2-y%5E2)-(1/3)*exp(-(x+1)%5E2-y%5E2);visible=true;umin=-4;umax=4;vmin=-4;vmax=4;grid=30;format=normal;alpha=-1;hidemyedges=true;constcol=rgb(255,0,0);view=0;contourcolor=red;fixdomain=false;contourplot=true;showcontourplot=false;firstvalue=-1;stepsize=0.2;numlevels=11;list=;uselist=false;xnum=46;ynum=46;show2d=false;hidesurface=false;hidelabels=true;showprojections=false;surfacecontours=true;projectioncolor=rgba(255,0,0,1);showxygrid=false;showxygridonbox=false;showconstraint=false&type=window;hsrmode=3;nomidpts=true;anaglyph=-1;center=-5.2487653277286155,6.815843602039553,5.098503557610455,1;focus=0,0,0,1;up=0.39284920127083023,-0.3373981166615778,0.8554718089651412,1;transparent=false;alpha=140;twoviews=false;unlinkviews=false;axisextension=0.7;xaxislabel=x;yaxislabel=y;zaxislabel=z;edgeson=true;faceson=true;showbox=true;showaxes=true;showticks=true;perspective=true;centerxpercent=0.5;centerypercent=0.5;rotationsteps=30;autospin=true;xygrid=false;yzgrid=false;xzgrid=false;gridsonbox=true;gridplanes=false;gridcolor=rgb(128,128,128);xmin=-4;xmax=4;ymin=-4;ymax=4;zmin=-4;zmax=4;xscale=2;yscale=2;zscale=2;zcmin=-8;zcmax=8;xscalefactor=1;yscalefactor=1;zscalefactor=1;tracemode=0;keep2d=false;zoom=0.89)

In [65]:
# Let's try to randomly generate several random inputs to the objective function
# and manually observe how the outputs change with different inputs

X = np.random.uniform(lower_bounds, upper_bounds)
value = objective_function(X)
print('objective_function(%.3f, %.3f) = %.3f' % (X[0], X[1], value))

objective_function(2.883, -2.909) = -0.000


# Part A: Hill-climbing

In [76]:
# TODO: Hill-climbing search algorithm: a loop that continually moves in the direction of increasing value.
# It terminates when it reaches a “peak” where no neighbor has a higher value.
#   objective function: function to be maximized
#   initial_state: initial (x, y) vector
#   step_size: numerical interval by which to change the current (x,y) state to generate a new neighboring state
#   print_iters: set to True to print out the current value at each iteration
#   returns: best [x, y] solution found
def hill_climbing(objective_function, initial_state=np.array([1, 0]), step_size=0.01, print_iters=True):
    # set the starting point of the search algorithm
    current_state = initial_state

    # loop until a peak is found
    i = 0
    while True:
        current_val = objective_function(current_state)
        neighbors = []
        x = float(current_state[0])
        y = float(current_state[1])

        neighbors.append([x+step_size, y])
        neighbors.append([x, y+step_size])
        neighbors.append([x+step_size, y+step_size])
        neighbor_vals = [objective_function(neighbor) for neighbor in neighbors]

        best_neighbor = neighbors[np.argmax(neighbor_vals)]
        best_neighbor_val = objective_function(best_neighbor)

        #print("BEST VAL: ", best_neighbor_value)

        if best_neighbor_val > objective_function(current_state):
            current_state = best_neighbor
        else:
            if print_iters:
                print('iteration: {}, current_state: {}, current_value: {}'.format(i, current_state,
                                                                                   objective_function(current_state)))
            return current_state

        i += 1

In [77]:
hill_climbing_solution = hill_climbing(objective_function)
print('Hill climbing solution is:', hill_climbing_solution)
print(hill_climbing_solution)


iteration: 29, current_state: [1.2900000000000003, 0.0], current_value: 3.592256699266626
Hill climbing solution is: [1.2900000000000003, 0.0]
[1.2900000000000003, 0.0]


# Part B: Random Restart

In [78]:
# TODO: Improvement to the Hill-climbing search algorithm using random restarts
#   objective function: function to be maximized
#   lower_bounds: minimum allowable values for the input vector to the objective function 
#   upper_bounds: maximum allowable values for the input vector to the objective function
#   step_size: numerical interval by which to change the current (x,y) state to generate a new neighboring state
#   num_restarts: how many times to restart hill-climbing
#   returns: best [x, y] solution found
def random_restart_hill_climbing(objective_function, lower_bounds, upper_bounds, step_size = 0.01, num_restarts=10):
    i = 0
    hill_climbing(objective_function)
    best_neighbors = []
    while i < num_restarts:
        x = random.uniform(lower_bounds[0], upper_bounds[0])
        y = random.uniform(lower_bounds[0], upper_bounds[0])
        sol = hill_climbing(objective_function, initial_state=[x, y])
        i+=1    
    return sol

In [79]:
random_restart_solution = random_restart_hill_climbing(objective_function, lower_bounds, upper_bounds)
print('Random restart hill climbing solution is:', random_restart_solution)

iteration: 29, current_state: [1.2900000000000003, 0.0], current_value: 3.592256699266626
iteration: 30, current_state: [3.581220589242987, -0.016763797853355585], current_value: 0.0012373723462853795
iteration: 584, current_state: [3.3301693672529424, -0.013562615149349888], current_value: 0.005628450341110371
iteration: 197, current_state: [3.397584335178382, -0.015719946478712894], current_value: 0.003800277837185901
iteration: 0, current_state: [2.111108755274751, 2.3637967796278394], current_value: 0.03595582245050744
iteration: 93, current_state: [-0.005070508430558802, 3.2365485422642797], current_value: 0.10024068469535546
iteration: 0, current_state: [2.4677557821575737, 0.20072596390555475], current_value: 0.3197920892984255
iteration: 597, current_state: [3.3340509596058916, -0.017919570059025232], current_value: 0.005504187940698846
iteration: 573, current_state: [3.3355808287953144, -0.012112752279542243], current_value: 0.005455789146984583
iteration: 0, current_state: [0

# Part C: Simulated Annealing

In [ ]:
# TODO: Simulated annealing algorithm
#   objective function: function to be maximized
#   lower_bounds: minimum allowable values for the input vector to the objective function 
#   upper_bounds: maximum allowable values for the input vector to the objective function
#   returns: best [x, y] solution found

def simulated_annealing(objective_function, lower_bounds, upper_bounds):
  return None


In [ ]:
simulated_annealing_solution = simulated_annealing(objective_function, lower_bounds, upper_bounds)
print('Simulated annealing solution is:', simulated_annealing_solution)